In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/4995/project_data.zip" "/content/project_data.zip"

In [3]:
!mkdir -p "/content/data"

In [4]:
!unzip -q "/content/project_data.zip" -d "/content/data"

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split, Subset
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
# Load the dataset
dataset = datasets.ImageFolder('/content/data/project_data', transform=transform)

# Assuming each class has 6000 images and they are evenly distributed
images_per_class = 6000
train_size_per_class = 4000
val_size_per_class = 1000
test_size_per_class = 1000

# Indices for splitting
train_indices = []
val_indices = []
test_indices = []

for i in range(len(dataset.classes)):
    start_idx = i * images_per_class
    indices = np.arange(start_idx, start_idx + images_per_class)
    np.random.shuffle(indices)

    train_indices.extend(indices[:train_size_per_class])
    val_indices.extend(indices[train_size_per_class:train_size_per_class + val_size_per_class])
    test_indices.extend(indices[train_size_per_class + val_size_per_class:])

# Creating subsets for train, val, and test
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

16000 4000 4000


In [9]:
model = models.resnet152(pretrained=True)

# Replace the last fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 classes for seasons

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:02<00:00, 104MB/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
best_val_accuracy = 0.0
best_model_path = 'best_model_resnet152.pth'
patience = 5
patience_counter = 0
num_epochs = 10

In [11]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy*100}%')

    # Check if model improved
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)
        print(f'Model saved as validation accuracy improved to {val_accuracy*100}%')
        patience_counter = 0  # reset counter if model improved
    else:
        patience_counter += 1
        print(f'No improvement in validation accuracy for {patience_counter} epochs.')

    # Early stopping check
    if patience_counter >= patience:
        print(f'Stopping training early as there has been no improvement for {patience} epochs.')
        break
print('Finished Training')

Epoch 1/10, Loss: 1.0378029011487961
Epoch 1/10, Validation Accuracy: 61.724999999999994%
Model saved as validation accuracy improved to 61.724999999999994%
Epoch 2/10, Loss: 0.9138526438474656
Epoch 2/10, Validation Accuracy: 64.275%
Model saved as validation accuracy improved to 64.275%
Epoch 3/10, Loss: 0.8566130851507187
Epoch 3/10, Validation Accuracy: 66.5%
Model saved as validation accuracy improved to 66.5%
Epoch 4/10, Loss: 0.8482947000265122
Epoch 4/10, Validation Accuracy: 66.175%
No improvement in validation accuracy for 1 epochs.
Epoch 5/10, Loss: 0.8180317846536637
Epoch 5/10, Validation Accuracy: 67.60000000000001%
Model saved as validation accuracy improved to 67.60000000000001%
Epoch 6/10, Loss: 0.8063187952041626
Epoch 6/10, Validation Accuracy: 67.77499999999999%
Model saved as validation accuracy improved to 67.77499999999999%
Epoch 7/10, Loss: 0.7828982535004616
Epoch 7/10, Validation Accuracy: 61.95%
No improvement in validation accuracy for 1 epochs.
Epoch 8/10, 

In [12]:
model.load_state_dict(torch.load(best_model_path))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total}%')

Accuracy on the test images: 68.35%
